In [1]:
from shapely.ops import cascaded_union
from ast import literal_eval
import pandas as pd
import geopandas as gp
data_dir = '../data/'
osm_data_dir = data_dir + 'osm/'
category_in_dc_path = osm_data_dir +'osm_category_in_dc.txt'

FRSQ_DIR ='../data/frsq/'
FRSQ_GRAPH_DIR = FRSQ_DIR+'graphs/'
SUFFIX = '_nearby'
frsq_venues_path = FRSQ_DIR+ 'frsq_extracted_venues{}.geojson'.format(SUFFIX)
frsq_venues_mapped_path = FRSQ_DIR+ 'frsq_extracted_venues_mapped{}.geojson'.format(SUFFIX)

In [2]:
category_in_dc_df = pd.read_csv(category_in_dc_path,index_col=0)
category_in_dc_df.category = category_in_dc_df.category.apply(eval)
exclude = 'exclude'
category_in_dc_df['has_category'] = category_in_dc_df.category.apply(lambda x: (len(x)>0) and (exclude not in x))
# category_in_dc_df[category_in_dc_df.category==set(['exclude'])]
category_in_dc_df[category_in_dc_df.has_category].shape,\
category_in_dc_df[category_in_dc_df.category.apply(len)>0].shape

((33003, 4), (35284, 4))

In [3]:
import sys, os
sys.path.insert(0, os.path.abspath('../'))
from utils.osm_helper import *
import geopandas as gp
from osmread import Node, Way, Relation
import datetime
import pandas as pd
OSM_DC_BBOX_PATH = '../data/osm/osm_dc_bbox.osm'
OSM_DC_BBOX = osm_container(OSM_DC_BBOX_PATH)
print OSM_DC_BBOX.data_size()


begin reading osm 2016-12-02 09:26:45.088000
finish reading osm 2016-12-02 09:27:54.617000
["len of <class 'osmread.elements.Way'> = 306332", "len of <class 'osmread.elements.Relation'> = 3427", "len of <class 'osmread.elements.Node'> = 2546260"]


In [4]:
def keep_obj(osm_objs, df,osmtype, osmtype_str):
    osm_ids = set(df[(df.type==osmtype_str) & (df['has_category']==True)].id.values)
    return [obj for obj in osm_objs.osm_objs[osmtype] if obj.id in osm_ids]

nodes_with_category = keep_obj(OSM_DC_BBOX, category_in_dc_df,Node, 'Node')
ways_with_category = keep_obj(OSM_DC_BBOX, category_in_dc_df,Way, 'Way')
rltns_with_category = keep_obj(OSM_DC_BBOX, category_in_dc_df,Relation, 'Relation')
len(nodes_with_category), len(ways_with_category), len(rltns_with_category)

(11769, 20017, 1217)

### transform foursquare, OSM data into geopandas

In [5]:
sys.path.insert(0, os.path.abspath('../../'))
from Cycling_Safe.utils.osm.osm2shp import *

#### resolve osm relation

In [6]:
rltn_pts = []
rltn_lns = []

for r in rltns_with_category:
    shps = rltn2dictShp(OSM_DC_BBOX,r)
    cnt = 0
    pts = [(r.id,'{}_{}'.format(r.id,i+cnt), r.tags, pt, 'Relation') for i, pt in enumerate(shps['Point'])]
    rltn_pts.extend(pts)
    cnt+=len(shps['Point'])
    lns = [(r.id,'{}_{}'.format(r.id,i+cnt), r.tags, pt, 'Relation') for i, pt in enumerate(shps['LineString'])]
    rltn_lns.extend(lns)
    cnt+=len(shps['LineString'])
    polys = [(r.id,'{}_{}'.format(r.id,i+cnt), r.tags, pt, 'Relation') for i, pt in enumerate(shps['Polygon'])]
    rltn_lns.extend(polys)
    
rltn_pts = pd.DataFrame(rltn_pts, columns=['id','id_keep','tag','geometry','type']).merge(category_in_dc_df)
rltn_pts = rltn_pts.drop(['id','has_category'], axis=1)
rltn_pts.columns = ['id','tag','geometry','type','category']

rltn_lns = pd.DataFrame(rltn_lns, columns=['id','id_keep','tag','geometry','type']).merge(category_in_dc_df)
rltn_lns = rltn_lns.drop(['id','has_category'], axis=1)
rltn_lns.columns = ['id','tag','geometry','type','category']


#### get pts

In [7]:
pts_bfr = 5

In [8]:
pts = [(nd.id, nd.tags, node2pt(nd),'Node') for nd in nodes_with_category]
pts_gpdf = gp.GeoDataFrame(pts, columns=['id','tag', 'geometry','type'])

pts_gpdf = pts_gpdf.merge(category_in_dc_df).drop('has_category',axis=1)
pts_gpdf = pts_gpdf.append(rltn_pts,ignore_index=True)
pts_gpdf.tag = pts_gpdf.tag.apply(str)

pts_gpdf.crs = {'init': u'epsg:4326'}
pts_gpdf_crs = pts_gpdf.to_crs(epsg=3559)
pts_gpdf_crs.geometry = pts_gpdf_crs.buffer(pts_bfr)

pts_gpdf_bfr = pts_gpdf_crs.to_crs(epsg=4326)
pts_gpdf_bfr.columns=['id','tag','geometry','type','category']

In [9]:
venues_gpdf = gp.read_file(frsq_venues_mapped_path)

venues_gpdf_crs = venues_gpdf.to_crs(epsg=3559)
venues_gpdf_crs.geometry = venues_gpdf_crs.buffer(pts_bfr)

venues_gpdf_bfr = venues_gpdf_crs.to_crs(epsg=4326)
venues_gpdf_bfr = venues_gpdf_bfr.drop(['checkins','name','tips','users'],axis=1)
venues_gpdf_bfr['type']='fq'
venues_gpdf_bfr.columns=['tag','geometry','id','category','type']
venues_gpdf_bfr = venues_gpdf_bfr[venues_gpdf_bfr.category!='no category']
venues_gpdf_bfr.category = venues_gpdf_bfr.category.apply(lambda x: set([x]))

#### get lns or polys, append rltn lns

In [10]:
lns = [(way.id, way.tags, way2lineOrpoly(OSM_DC_BBOX, way),'Way') for way in ways_with_category]
lns_gpdf = gp.GeoDataFrame(lns, columns=['id','tag', 'geometry','type'])

lns_gpdf = lns_gpdf.merge(category_in_dc_df[category_in_dc_df.has_category]).drop('has_category', axis=1)
print lns_gpdf.shape

lns_gpdf = lns_gpdf.append(rltn_lns,ignore_index=True)
print lns_gpdf.shape

lns_gpdf.tag = lns_gpdf.tag.apply(str)

lns_gpdf.columns=['id','tag','geometry','type','category']

Ring Self-intersection at or near point -77.020059900000007 38.815362499999999


(20017, 5)
(23956, 5)


D:\���ѧϰ����\Life in Maryland\Research Project��CyclingSafe\github\Cycling_Safe\utils\osm\osm2shp.py:30: UserWarning: way id=59249848 is not valid as a <class 'shapely.geometry.polygon.Polygon'>
  warnings.warn("way id={} is not valid as a {}".format(way.id, type(obj)))


### find overlap

#### overlap between osm nodes and fq

In [103]:
fq_pts = gp.tools.sjoin(pts_gpdf_bfr, venues_gpdf_bfr)


AttributeError: 'GeoDataFrame' object has no attribute 'id'

In [113]:
fq_pts
# over kill example
fq_pts[(fq_pts.apply(lambda x: len(x.category_right - x.category_left)==0, axis=1))]
# fq_pts[(fq_pts.id_left==2276095966)]

,id_left,tag_left,geometry,type_left,category_left,index_right,tag_right,id_right,category_right,type_right
76,49734611,{u'highway': u'traffic_signals'},POLYGON ((-77.04331293640715 38.92800152138454...,Node,set([cycling facilities]),14338,Bike Rental / Bike Share,4e54f80ad4c0fe0342802d27,set([cycling facilities]),fq
333,49756895,{u'highway': u'crossing'},POLYGON ((-77.00198237099791 38.88519300099211...,Node,set([cycling facilities]),11518,Bike Rental / Bike Share,4e1c41e4887735dda340d508,set([cycling facilities]),fq
7103,2275495853,"{u'amenity': u'bicycle_rental', u'capacity': u...",POLYGON ((-77.00217137104065 38.88514010108537...,Node,set([cycling facilities]),11518,Bike Rental / Bike Share,4e1c41e4887735dda340d508,set([cycling facilities]),fq
11273,4111818602,"{u'craft': u'shoemaker', u'name': u'Shoe Repair'}",POLYGON ((-77.03810993380768 38.93121401881736...,Node,set([retail shop]),13576,Laundry Service,505b4a96e4b03f672966fe16,set([retail shop]),fq
451,49765287,{u'highway': u'traffic_signals'},POLYGON ((-77.02423242966984 38.93632631197021...,Node,set([cycling facilities]),8882,Bike Rental / Bike Share,4d330ded6c7c721e7fd6b556,set([cycling facilities]),fq
556,49773638,"{u'source': u'survey', u'highway': u'crossing'}",POLYGON ((-77.02767263509807 38.92961551366762...,Node,set([cycling facilities]),13780,Bike Rental / Bike Share,4f84b7c4e4b015c04257f560,set([cycling facilities]),fq
3404,603190933,"{u'dcgis:pubdate': u'2009-11-13', u'dcgis:gis_...",POLYGON ((-77.06525415036059 38.91075283221036...,Node,set([transportation]),19206,Bus Station,4f9c3f62e4b020e60f9d2dfc,set([transportation]),fq
7077,2174502624,"{u'amenity': u'restaurant', u'name': u'Grill K...",POLYGON ((-77.06590475480317 38.90525643253143...,Node,set([food]),1759,Indian Restaurant,4726def6f964a520db4b1fe3,set([food]),fq
5121,741670744,"{u'addr:housenumber': u'1351', u'opening_hours...",POLYGON ((-77.06340315265429 38.90791393129712...,Node,set([retail shop]),413,Tailor Shop,4aa569fcf964a5203e4820e3,set([retail shop]),fq
1221,60471261,"{u'addr:housenumber': u'1847', u'cuisine': u'C...",POLYGON ((-77.03164384657625 38.91542051562708...,Node,"set([food, nightlife spot])",17341,American Restaurant,3fd66200f964a520d3f11ee3,set([food]),fq


In [12]:
fq_pts_overlap = fq_pts[fq_pts.apply(lambda x: len(x.category_right - x.category_left)==0, axis=1)]
fq_pts_overlap.shape

(954, 10)

In [13]:
overlap_nodeid = np.unique(fq_pts_overlap.id_left)
overlap_fqid = np.unique(fq_pts_overlap.id_right)
overlap_nodeid.shape, overlap_fqid.shape

((807L,), (795L,))

In [14]:
pts_not_overlap = pts_gpdf_bfr[~pts_gpdf_bfr.id.isin(overlap_nodeid)]
fqs_not_overlap = venues_gpdf_bfr[~venues_gpdf_bfr.id.isin(overlap_fqid)]

In [15]:
overlap_merged = []
for oid, grp in fq_pts_overlap.groupby('id_left'):
    pts = [grp.geometry.values[0]]
    for fqid in grp.id_right.values:
        pts.append(venues_gpdf_bfr[venues_gpdf_bfr.id==fqid].geometry.values[0])
    tag = str(grp.tag_left.values[0])+'<br>'+ '<br>'.join(grp.tag_right.values)
    category = grp.category_left.values[0]
    union_pts = cascaded_union(pts)
    t = grp.type_left.values[0]+'_fq'
    overlap_merged.append((oid, tag, category, union_pts,t))
overlap_merged = gp.GeoDataFrame(overlap_merged,columns=['id','tag','category','geometry','type'])

In [16]:
pts_merged = pd.concat([pts_not_overlap, fqs_not_overlap, overlap_merged],ignore_index=True)

#### overlap between osm ways

In [17]:
from Cycling_Safe.utils.geofunc import remove_equal_shpobj, merge_within

In [18]:
def get_sub_tree(tree_df, node_id, looping=0):
    mask_node = (tree_df.node == node_id)
    mask_children = (tree_df.parent == node_id)
    assert looping < 100, 'looping too much nodeid:' + str(node_id)

    node = tree_df[mask_node]
    nid = node.node.values[0]
    subtree = {nid}

    children_id = tree_df[mask_children].node.values
    if len(children_id) == 0:
        return subtree, node.shape
    
    subtree.update(children_id)

    for cid in children_id:
        c_subtree, c_tree_shape = get_sub_tree(tree_df, cid, looping + 1)
        subtree.update(c_subtree)
    return subtree, node.shape

In [19]:
tree, tree_all_parent, tree_direct_parent, equal_pair_index = merge_within(lns_gpdf)
tree = tree.apply(lambda x: x)

begin merge within 2016-12-02 09:28:33.834000
keep = 23319 equal pair = 637 (23319, 5) 2016-12-02 09:28:43.530000
sjoin.shape = (34072, 10) 2016-12-02 09:29:25.551000
messy tree shape = (34072, 2) 2016-12-02 09:29:25.578000
clean tree shape = (23319, 3) 2016-12-02 09:29:38.614000


In [20]:
df_epi = pd.DataFrame(equal_pair_index,columns=['keep','equal'])

In [21]:
nodes_have_children = tree_direct_parent[
    (tree_direct_parent.node.isin(tree_direct_parent.parent.value_counts().index.tolist()))
    & (tree_direct_parent.parent == -1)].node.tolist()
print len(nodes_have_children)
print datetime.datetime.now()
sub_trees = []
shape_error = []
assert_error = []
for node_id in nodes_have_children:
    try:
        subtree, shape = get_sub_tree(tree_direct_parent, node_id)
    except Exception as e:
        assert_error.append((node_id, e))
        continue
    if shape != (1, 7):
        shape_error.append((node_id, subtree, shape))
    sub_trees.append(subtree)
print datetime.datetime.now()
print len(assert_error)
sub_trees = dict(zip(nodes_have_children, sub_trees))

1209
2016-12-02 09:29:38.662000
2016-12-02 09:29:52.212000
0


In [22]:
new_category = []
for idx, a in tree.iterrows():
    equal_objs = lns_gpdf.loc[df_epi[df_epi.keep==idx].equal].category.values
    c = set.union(*equal_objs) if len(equal_objs)>0 else set()
    c.update(a.category)
    if not idx in sub_trees:
        new_category.append(c)
    else:
        subtree = sub_trees[idx]
        c.update(set.union(*lns_gpdf.loc[list(subtree)].category.values))
        new_category.append(c)


In [23]:
tree['category'] = new_category

In [24]:
lns_merged = gp.GeoDataFrame(tree)

lns_merged.crs={'init': 'epsg:4326', 'no_defs': True}
lns_merged.shape

(14554, 5)

####  old way

In [37]:
# lns_within = gp.tools.sjoin(lns_gpdf, lns_gpdf,op='within')
# lns_contains = gp.tools.sjoin(lns_gpdf, lns_gpdf,op='contains')

# lns_within_exclude_self = lns_within[lns_within.id_left!=lns_within.id_right]
# lns_contains_exclude_self = lns_contains[lns_contains.id_left!=lns_contains.id_right]
# lns_equal_shps = lns_within_exclude_self.merge(lns_contains_exclude_self, left_on=['id_left','id_right'], right_on=['id_left','id_right'])

# # it's ok if they are equal but with different categories
# print lns_equal_shps[lns_equal_shps.category_left_x!=lns_equal_shps.category_right_x].shape

# equal_pairs = lns_equal_shps.apply(lambda x: tuple(sorted([x.id_left, x.id_right])), axis=1).value_counts()
# equal_pairs = equal_pairs[equal_pairs>1]
# equal_retrieve = [x[0] for x in equal_pairs.index.values]
# print len(equal_retrieve)

# lns_not_contained = lns_gpdf[~lns_gpdf.id.isin(lns_contains_exclude_self.id_right)]
# print lns_not_contained.shape
# lns_merged = lns_not_contained.append(lns_gpdf[lns_gpdf.id.isin(equal_retrieve)],ignore_index=True)
# print lns_merged.shape
# lns_merged.crs={'init': 'epsg:4326', 'no_defs': True}


(148, 18)
678
(14420, 5)
(15057, 5)


####  overlap between ways and points

In [25]:
lns_pts_overlap = gp.tools.sjoin(lns_merged, pts_merged)
# lns_pts_overlap = lns_pts_overlap[lns_pts_overlap.category_left==lns_pts_overlap.category_right]
lns_pts_overlap = lns_pts_overlap[lns_pts_overlap.apply(lambda x: len(x.category_right - x.category_left)==0, axis=1)]
pts_in_lns = pd.unique(lns_pts_overlap.index_right)
pts_not_in_lns = pts_merged.drop(pts_in_lns)

### visualize data on the map

In [34]:
lns_merged_vis = lns_merged.copy()
lns_merged_vis['color'] = lns_merged_vis.apply(lambda x: {'Way':'#00F','Relation':'#0FF'}[x.type], axis=1)
lns_merged_vis.category = lns_merged_vis.category.apply(str)

pts_not_in_lns_vis = pts_not_in_lns.copy()
pts_not_in_lns_vis['color'] = pts_not_in_lns_vis.apply(lambda x: {'Node':'#F00', 'fq':'#0F0', 'Node_fq':'#FF0','Relation_fq':'#FF0', 'Relation':'#0FF'}[x.type],axis=1)
pts_not_in_lns_vis.category = pts_not_in_lns_vis.category.apply(str)

import utils.leaflet_creation_v2;reload(utils.leaflet_creation_v2)
from utils.leaflet_creation_v2 import get_color_for_df, create_map_visualization
html_title = 'osm_frsq'
file_path = '../data/'
file_name = 'osm_frsq'
lon, lat = -77.0163424758, 38.9047829846
zoom = 12
init_layer = ['light']
map_layer = ['light','streets','satellite']
binding_data = [['pts', 'points'], ['osm_lns','osm lines']]
gpdfs = [pts_not_in_lns_vis, lns_merged_vis]
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layer, map_layer, binding_data, gpdfs)

## output category distribution

### get data near segment

In [69]:
def crs_prepossess(gpdf, init_crs, bfr_crs):
    gpdf_crs = gpdf.copy()
    if gpdf_crs.crs==None:
        gpdf_crs.crs = {'init': u'epsg:{}'.format(init_crs)}
    return gpdf_crs.to_crs(epsg=bfr_crs)

def bfr_20m(seg):
    return seg.geometry.buffer(20)

def get_objs_nearby(segments, objs, bfr_func, init_crs, bfr_crs):
    seg_crs = crs_prepossess(segments, init_crs, bfr_crs)
    obj_crs = crs_prepossess(objs, init_crs, bfr_crs)
    seg_crs.geometry = seg_crs.apply(bfr_20m,axis=1)

    sjoin = gp.tools.sjoin(seg_crs, obj_crs,op='intersects')
    obj_ids_nearby = set(sjoin.index_right)
    objs_nearby = objs[objs.index.isin(obj_ids_nearby)]
    return sjoin,objs_nearby


In [70]:
dc_segments = gp.read_file('../data/opendc_segments.geojson')
semantic_data = lns_merged.append(pts_not_in_lns,ignore_index=True)
semantic2seg, semantic_near = get_objs_nearby(dc_segments, semantic_data, bfr_20m, 4326, 3559)

In [71]:
semantic2seg.shape, semantic_near.shape

((95087, 27), (31722, 5))

### output category count distribution

In [50]:
list_mapped = semantic_near.category.values
len(list_mapped)

31722

In [51]:
from collections import defaultdict
def get_sum_categories(list_mapped):
    sum_categories = defaultdict(int)
    for mapped in list_mapped:
        for c in list(mapped):
            sum_categories[c]+=1
    return sum_categories
pd.DataFrame(get_sum_categories(list_mapped).items(),columns=['category','osm_plus_fq']).to_csv('../data/category_osm_fq.csv')

### output category segment distribution

In [95]:
semantic_per_segment = {}
for segid, grp in semantic2seg.groupby('STREETSEGID'):  
#     print segid
    semantic_per_segment[segid] = dict(get_sum_categories(grp.category.values))
len(semantic_per_segment), dc_segments.shape

(12522, (13522, 22))

In [96]:
dc_segment_category = dc_segments[['STREETSEGID']].merge(pd.DataFrame(semantic_per_segment.items(),columns=['STREETSEGID','category']),how='left')

In [97]:
dc_segment_category['total'] = dc_segment_category.category.apply(lambda x: sum(x.values()) if type(x)!=float else 0)

In [98]:
labels = ['art','outdoors and recreation', 'retail shop', 'professional service', 'food', 'nightlife spot','residence','schools&university','cycling facilities','transportation']
for l in labels:
    dc_segment_category[l] = dc_segment_category.category.apply(lambda x: x.get(l,0) if type(x)!=float else 0)

In [101]:
dc_segment_category.drop('category',axis=1).to_csv('../data/category_osm_fq_per_segment.csv')

In [ ]:
lns_gpdf_vis = lns_gpdf
lns_gpdf_vis['color'] = '#00F'
pts_gpdf_vis = pts_gpdf_crs.to_crs(epsg=4326)
pts_gpdf_vis['color'] = '#F00'
venues_gpdf_vis = venues_gpdf_crs.to_crs(epsg=4326)
venues_gpdf_vis['color'] = '#0F0'

import utils.leaflet_creation_v2;reload(utils.leaflet_creation_v2)
from utils.leaflet_creation_v2 import get_color_for_df, create_map_visualization
html_title = 'osm_pts_frsq'
file_path = '../data/'
file_name = 'osm_pts_frsq'
lon, lat = -77.0163424758, 38.9047829846
zoom = 12
init_layer = ['light']
map_layer = ['light','streets','satellite']
binding_data = [['fq', 'foursqaure venues'],['osm_pts', 'osm points'], ['osm_lns','osm lines']]
gpdfs = [venues_gpdf_vis, pts_gpdf_vis, lns_gpdf_vis]
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layer, map_layer, binding_data, gpdfs)